In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding
from keras.datasets import mnist
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 문장을 잘라줌.
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 크롤링을 통한 네이버 영화 평점 및 평가글 등 데이터를 가져옴.
df = pd.read_csv('data/naver_star_data.csv')
df.head()

# 평점 및 평가글만 가져옴.
# 253691개의 데이터.
df2 = df[['평점', '평가글']]
df2.head()

,평점,평가글
0,1.0,"B급 이하전편보다 퇴보된 CG, 중구난방식 연출, 러닝타임 늘리기 위한 쓸모없는 컷..."
1,3.0,이딴 영화가 평점 8점 후반대라는 게 믿기지 않는다. 역시 네이버 평점은 믿고 걸러...
2,6.0,역시 원더우먼 영화는 주연배우 갤가돗과 크리스파인이 다 살리네.. 감독은 확실히 영...
3,2.0,진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ
4,1.0,히어로물의 액션을 기대했음. 그러나 졸렬한 액션과 이상한 전개로 마지막 영화관을 나...


In [3]:
# 결측치 제거.
df2.isna().sum()
df2.dropna(inplace=True)

# 평점이 5점 이하는 0으로 변환.
a1 = df2.query('평점<=5').index
df2.loc[a1, '평점'] = 0

# 평점이 5점 이상은 1로 변환.
a1 = df2.query('평점>=5').index
df2.loc[a1, '평점'] = 1

# 평가글 추출.
docs = df2['평가글'].values
docs

# 결과 추출,
classes = df2['평점'].values
classes 

# 토큰화.
token = Tokenizer()
token.fit_on_texts(docs)

# 단어 사전을 통해 문장의 각 단어를 숫자로 변환.
x = token.texts_to_sequences(docs)
len(x)

253691

In [4]:
# 한 문장의 최대 단어 수를 가져옴.
max_cnt = 0
for c in x :
    # 현재 문장의 글자수.
    cnt = len(c)
    # 현재 문장의 글자수가 이전 최대 수치보다 많으면 덮어씌움.
    if max_cnt < cnt :
        max_cnt = cnt
print('최대 단어의 개수 : ', max_cnt)

최대 단어의 개수 :  48


In [5]:
# padding => 서로 길이가 다른 리스트의 개수를 max_cnt로 맞춰줌.
padded_x = pad_sequences(x, max_cnt)
padded_x

array([[     0,      0,      0, ...,     56,   9293,  17911],
       [     0,      0,      0, ...,  63776,    231, 101809],
       [     0,      0,      0, ...,  25328,     90,    422],
       ...,
       [     0,      0,      0, ..., 383505, 383506,  27644],
       [     0,      0,      0, ...,   1432,     37,    239],
       [     0,      0,      0, ...,  12420, 383512,   7694]])

In [6]:
# 단어의 수를 파악.
word_size = len(token.word_index) + 1

# 학습 모델을 구성.
model = Sequential()
model.add(Embedding(word_size, 8, input_length=max_cnt))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# 컴파일.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습.
model.fit(padded_x, classes, epochs=20 )

Epoch 1/20
7928/7928 [==============================] - 145s 18ms/step - loss: 0.3074 - accuracy: 0.8806
Epoch 2/20
7928/7928 [==============================] - 152s 19ms/step - loss: 0.1567 - accuracy: 0.9397
Epoch 3/20
7928/7928 [==============================] - 159s 20ms/step - loss: 0.1039 - accuracy: 0.9627
Epoch 4/20
7928/7928 [==============================] - 162s 20ms/step - loss: 0.0700 - accuracy: 0.9766
Epoch 5/20
7928/7928 [==============================] - 150s 19ms/step - loss: 0.0490 - accuracy: 0.9844
Epoch 6/20
7928/7928 [==============================] - 151s 19ms/step - loss: 0.0352 - accuracy: 0.9887
Epoch 7/20
7928/7928 [==============================] - 139s 17ms/step - loss: 0.0252 - accuracy: 0.9919
Epoch 8/20
7928/7928 [==============================] - 138s 17ms/step - loss: 0.0187 - accuracy: 0.9940
Epoch 9/20
7928/7928 [==============================] - 138s 17ms/step - loss: 0.0143 - accuracy: 0.9955
Epoch 10/20
7928/7928 [==============================] 

In [7]:
# 정확도.
model.evaluate(padded_x, classes)[1]

7928/7928 [==============================] - 11s 1ms/step - loss: 0.0037 - accuracy: 0.9988


0.9987662434577942

In [8]:
# Test 문장 생성.
docs = [
    '이 영화 재미있어요',
    '이 영화 재미 없어요',
    '추천 드립니다',
    '추천 드리지 않습니다'
]

In [9]:
# 예측.
# 1 : 긍정, 0 : 부정
x = token.texts_to_sequences(docs)
padded_x = pad_sequences(x, max_cnt)
pred = (model.predict(padded_x) > 0.5).astype('int32')
pred

array([[1],
       [0],
       [1],
       [0]])